# Exacting Strike

**The question we want to answer:**
*"I'm a level 5 fighter with a +1 Striking Maul. What's my average damage? Is Exacting Strike a good feat?"*

Let's simulate striking 3 times in a round, against a target that is not off-guard.
On your second strike, you _may_ use Exacting Strike. If you do, the outcome changes the MAP on your third strike.
Every time you roll a critical hit, the target needs to pass a fortitude save or be knocked prone.
If they're knocked prone, they become flat-footed to the following attacks, and they will have to stand on the next round, provoking a Reactive Strike.

In [ ]:
import hvplot.xarray
import numpy as np
import panel
import xarray

import pathfinder2e_stats as pf2

np.random.seed(0)
hvplot.extension("matplotlib")

### Targets
Let's define three targets:

- a mook that's 2 levels below us, low AC, and low fortitude (GMC pp. TODO)
- a monster at our same level, medium AC, and medium fortitude
- a boss that's 2 levels above us, high AC, and high fortitude

In [ ]:
targets = xarray.Dataset(
    data_vars={
        "AC": ("target", [16, 21, 25]),
        "fortitude": ("target", [6, 12, 18]),
    },
    coords={"target": ["level 3 low", "level 5 medium", "level 7 high"]},
)
targets.to_pandas()

### Attacker
Then we define our own stats.

**Note:** the same way we defined multiple targets using an xarray.Dataset, we could have multiple attackers, for example a fighter vs. a barbarian.

In [ ]:
attack_bonus = 5 + 6 + 4 + 1  # Level 5, master, STR+4, item+1
class_DC = 10 + 5 + 2 + 4  # Level 5, trained, STR+4
attack_bonus, class_DC

Let's create a damage profile for our weapon.

In [ ]:
damage_spec = pf2.Damage("bludgeoning", 2, 12, 4)

### First strike

In [ ]:
strike1 = pf2.damage(
    pf2.check(attack_bonus, DC=targets.AC),
    damage_spec,
)
strike1

The target must roll a Fortitude save after every critical hit or be knocked prone.
Let's pre-roll them in advance as the bonus and DC don't change.

In [ ]:
fort_saves = pf2.check(
    bonus=targets.fortitude,
    DC=class_DC,
    allow_critical_success=False,
    allow_critical_failure=False,
    dims={"strike": 4},
)
fort_saves.coords["strike"] = [1, 2, 3, "reactive"]
knocked_prone = fort_saves.outcome == pf2.DoS.failure
knocked_prone.mean("roll").round(2).T.to_pandas()

Calculate the chance of being knocked prone by the first strike

In [ ]:
knocked_prone_1 = knocked_prone.sel(strike="1", drop=True).where(
    strike1.outcome == pf2.DoS.critical_success, False
)
knocked_prone_1.mean("roll").round(2).to_pandas()

### Second strike
The AC changes depending if the target has been knocked prone by the first strike or not.

In [ ]:
strike2 = pf2.damage(
    pf2.check(attack_bonus - 5, DC=targets.AC - knocked_prone_1 * 2),
    damage_spec,
)
knocked_prone_2 = (
    knocked_prone.sel(strike="2", drop=True).where(
        strike2.outcome == pf2.DoS.critical_success, False
    )
) | knocked_prone_1
knocked_prone_2.mean("roll").round(2).to_pandas()

### Third strike
We want to investigate the benefit of Exacting Strike. So, from now on we're going to roll everything twice, with and without the feat.

In [ ]:
MAP3 = xarray.concat(
    [xarray.DataArray(10), xarray.where(strike2.outcome == pf2.DoS.failure, 5, 10)],
    dim="exacting_strike",
)
MAP3.coords["exacting_strike"] = [False, True]

strike3 = pf2.damage(
    pf2.check(attack_bonus - MAP3, DC=targets.AC - knocked_prone_2 * 2),
    damage_spec,
)
knocked_prone_3 = (
    knocked_prone.sel(strike="3", drop=True).where(
        strike3.outcome == pf2.DoS.critical_success, False
    )
) | knocked_prone_2
knocked_prone_3.mean("roll").round(2).to_pandas()

And finally the reactive strike, which happens only if the target is prone by the end of the round

In [ ]:
reactive_strike_check = pf2.check(attack_bonus, DC=targets.AC)
reactive_strike_check["outcome"] = reactive_strike_check.outcome.where(
    knocked_prone_3, pf2.DoS.no_roll
)
reactive_strike = pf2.damage(reactive_strike_check, damage_spec)

knocked_prone_by_reactive_strike = knocked_prone.sel(
    strike="reactive", drop=True
).where(reactive_strike.outcome == pf2.DoS.critical_success, False)
knocked_prone_by_reactive_strike.mean("roll").round(2).to_pandas()

We're done! Let's assemble our aggregated object.

In [ ]:
all_strikes = xarray.concat([strike1, strike2, strike3, reactive_strike], dim="strike")
all_strikes.coords["strike"] = fort_saves.strike
all_strikes["prone_at_end_of_round"] = knocked_prone_3
all_strikes["prone_on_reactive_strike"] = knocked_prone_by_reactive_strike
all_strikes

We can finally aggregate our measures to gather insights.

In [ ]:
agg_measures = all_strikes.sum("strike").mean("roll")[
    ["total_damage", "prone_at_end_of_round", "prone_on_reactive_strike"]
]
agg_measures["any_damage"] = (all_strikes["total_damage"].sum("strike") > 0).mean(
    "roll"
)
agg_measures.stack(idx=["target", "exacting_strike"]).to_array("measure").round(
    2
).T.to_pandas()

What's the % benefit of exacting strike compared to three regular strikes?

In [ ]:
(
    agg_measures.sel(exacting_strike=True) / agg_measures.sel(exacting_strike=False)
).to_pandas()

The answer to the original question, _is Exacting Strike a good feat?_ is that it's quite inconsequential against weak enemies but, if you start your round in striking range of a boss and you've got nothing better to do with your third action, it will yield a solid 9% damage boost on average and will let you deal _some_ damage 6% more frequently.

In almost all cases it's inconsequential fort the purpose of triggering special abilities that go off on critical hits (you'd need to crit on a 19 on the die or less, while at MAP-5, for it to matter).


What's the damage distribution?

In [ ]:
w_exacting_strike = panel.widgets.Checkbox(name="Exacting Strike")

(
    all_strikes["total_damage"]
    .sum("strike")
    .to_dataset("target")
    .interactive.sel(exacting_strike=w_exacting_strike, drop=True)
    .hvplot.violin()
)

### Homework

- How does Exacting Strike perform compared to Vicious Swing?
- What's better, a sword (off-guard on a crit, no save) or a hammer (prone on a crit and trigger Reactive Strike, but with save)?
- How does a two-hander (e.g. maul) perform compared to two one-handers (e.g. warhammer and light hammer) with Double Slice?
- What's the damage distribution of a barbarian vs. that of a fighter?
- How much extra damage, on average, does a +1 to hit (or a -1 to AC) yield?

### Last words
In real play, circumstance is everything. For example, Exacting Strike is worthless when you have to spend one action moving into position (at least until you start getting Hasted with some consistency). Knocking a target prone is much more valuable if there are multiple martials with Reactive Strike, Stand Still, or similar feats in the party. Making a target off-guard is a lot more valuable if there's a rogue in party; etc. etc. 